# Updating Hosted Feature Layer using Excel Table

#### Ensure that input table (Excel) and target layer (hosted layer) share the same field names
#### The hosted layer is imported from ArcGIS Online.

Reference: https://developers.arcgis.com/python/samples/updating-features-in-a-feature-layer/

In [1]:
from pathlib import Path
import sys

from arcgis.gis import GIS, Item
from arcgis.features import FeatureLayerCollection
from arcgis.mapping import WebMap
from datetime import datetime
import arcpy
import os
import pandas as pd
import numpy as np

In [2]:
from arcgis.features.managers import FeatureLayerManager
from arcgis import features
from arcgis import geometry
from copy import deepcopy

In [55]:
# Define parameters
url_agol = "https://railway-sector.maps.arcgis.com"
username = "matsuzakieijinscrex"
password = "Gorilla@367*"
target_layer_itemid = "9b6d40d8bff04eca91c8b0f0fa48dd93" # viaduct associated feature layer (do not choose scene layer)
input_table = r"C:\Users\oc3512\Dropbox\01-Railway\02-NSCR-Ex\01-N2\03-During-Construction\02-Civil\03-Viaduct\01-Masterlist\02-Compiled\Viaduct_MasterList.xlsx"
join_field = 'uniqueID'
transfer_fields = ['Status1', 'end_date']

In [20]:
gis = GIS(url_agol, username, password)

In [21]:
featureLayer = gis.content.get(target_layer_itemid)
target_layer = featureLayer.layers[0]

In [22]:
#querying without any conditions returns all the features
target_fset = target_layer.query()
target_fset.sdf.head()

# First get all features
all_target_features = target_fset.features
all_target_features[0]

{"geometry": {"rings": [[[13445235.8498, 1676714.3137], [13445235.6773, 1676714.2939], [13445235.5045, 1676714.3133], [13445235.3405, 1676714.3709], [13445235.1933, 1676714.4638], [13445235.0702, 1676714.5873], [13445234.9778, 1676714.7352], [13445234.9201, 1676714.9003], [13445234.9005, 1676715.0739], [13445234.9198, 1676715.2477], [13445234.977, 1676715.4129], [13445235.0692, 1676715.561], [13445235.192, 1676715.6847], [13445235.339, 1676715.7779], [13445235.503, 1676715.8358], [13445235.6756, 1676715.8555], [13445235.8483, 1676715.8361], [13445236.0123, 1676715.7786], [13445236.1595, 1676715.6857], [13445236.2826, 1676715.5622], [13445236.3751, 1676715.4143], [13445236.4328, 1676715.2492], [13445236.4523, 1676715.0755], [13445236.433, 1676714.9018], [13445236.3759, 1676714.7367], [13445236.2836, 1676714.5885], [13445236.1608, 1676714.4648], [13445236.0138, 1676714.3716], [13445235.8498, 1676714.3137]]], "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"OBJEC

In [56]:
input_table = pd.read_excel(input_table)
input_table.head()

,Layer,PierNumber,Type,Status1,CP,idd,uniqueID,nPierNumber,end_date,ID,planned_date,updated
0,N-01_PILE,MT01-01,1,4,N-01,486.0,1,MT011A,2023-03-28,N01-STR-YN-10320,2023-04-14,2024-01-24
1,N-01_PILE,MT01-01,1,4,N-01,487.0,2,MT011B,2023-03-27,N01-STR-YN-10320,2023-04-14,2024-01-24
2,N-01_PILE,MT01-01,1,4,N-01,488.0,3,MT011C,2023-03-30,N01-STR-YN-10320,2023-04-14,2024-01-24
3,N-01_PILE,MT01-01,1,4,N-01,489.0,4,MT011D,2023-03-31,N01-STR-YN-10320,2023-04-14,2024-01-24
4,N-01_PILE,MT01-01,1,4,N-01,490.0,5,MT011E,2023-04-01,N01-STR-YN-10320,2023-04-14,2024-01-24


## Update both status and date fields (REFERENCE ONLY)
### XXXXX DO NOT USE THIS XXXXXX

In [13]:
# 1. Both status and date
for_update = []
uniqueIDs = [f for f in input_table[join_field]]

for unique_id in uniqueIDs[:10]:
    # Status for target
    original_feature = [f for f in all_target_features if f.attributes[join_field] == unique_id][0]
    to_be_updated = deepcopy(original_feature)
    
    # get the matching row from input_table
    #matching_uniqueid = input_table.where(input_table.uniqueID == unique_id).dropna()
    
    # get the status for input and target layers
    ## Input 
    id = input_table.index[input_table[join_field] == unique_id]
    status_input = input_table[transfer_fields[0]].loc[id].iloc[0]
    enddate_input = pd.to_datetime(input_table[transfer_fields[1]].loc[id].iloc[0]).date()
    
    print(unique_id)
    print(enddate_input)
    
    ## Target
    status_target = original_feature.attributes[transfer_fields[0]]
    objectid_target = original_feature.attributes['OBJECTID']
    
    try:
        timestamp = original_feature.attributes[transfer_fields[1]] / 1000
        enddate_target = datetime.fromtimestamp(timestamp).date()
    except:
        pass
    
    ## If input and target does not match in status and end_date, append to for_update list
    if status_input != status_target or enddate_input != enddate_target:
        to_be_updated = {
            'attributes': {
                'OBJECTID': objectid_target,
                transfer_fields[0] : status_input,
                transfer_fields[1] : enddate_input
            }
        }
        for_update.append(to_be_updated)

SyntaxError: invalid syntax (<string>, line 33)

In [259]:
if len(for_update) == 0:
    print('No need to update target feature layer')
else:
    target_layer.edit_features(updates=for_update)


No need to update target feature layer


# Update Status Field Only

In [24]:
for_update = []
uniqueIDs = [f for f in input_table[join_field]]

for unique_id in uniqueIDs:
    # Status for target
    original_feature = [f for f in all_target_features if f.attributes[join_field] == unique_id][0]
    to_be_updated = deepcopy(original_feature)
    
    # get the matching row from input_table
    #matching_uniqueid = input_table.where(input_table.uniqueID == unique_id).dropna()
    
    # get the status for input and target layers
    ## Input 
    id = input_table.index[input_table[join_field] == unique_id]
    status_input = input_table[transfer_fields[0]].loc[id].iloc[0]
    
    ## Target
    status_target = original_feature.attributes[transfer_fields[0]]
    objectid_target = original_feature.attributes['OBJECTID']
    
    ## If input and target does not match in status and end_date, append to for_update list
    if status_input != status_target:
        to_be_updated = {
            'attributes': {
                'OBJECTID': objectid_target,
                transfer_fields[0] : status_input,
            }
        }
        for_update.append(to_be_updated)

In [25]:
len(for_update)

253

In [26]:
if len(for_update) == 0:
    print('No need to update target feature layer')
else:
    target_layer.edit_features(updates=for_update)

# Update Date Field Only

In [84]:
for_update = []
uniqueIDs = [f for f in input_table[join_field]]

for unique_id in uniqueIDs:
    # Status for target
    original_feature = [f for f in all_target_features if f.attributes[join_field] == unique_id][0]
    to_be_updated = deepcopy(original_feature)
        
    # get the status for input and target layers
    ## Input 
    id = input_table.index[input_table[join_field] == unique_id]
    enddate_input = pd.to_datetime(input_table[transfer_fields[1]].loc[id].iloc[0]).date()
    
    temp = pd.to_datetime(input_table[transfer_fields[1]].loc[id])
    temp2 = np.isnat(temp)
    if temp2.bool() == False: # only if enddate_input is not empty, update the date
        to_be_updated = {
            'attributes': {
                'OBJECTID': objectid_target,
                transfer_fields[0] : enddate_input,
            }
        }
        for_update.append(to_be_updated)
    else:
        pass

In [85]:
len(for_update)

5194

In [86]:
if len(for_update) == 0:
    print('No need to update target feature layer')
else:
    target_layer.edit_features(updates=for_update)